# 작업 2유형
- https://www.datamanim.com/dataset/03_dataq/typetwo.html#id3

# 5. 중고차 가격 예측 데이터(Regression)

> Attention
- 데이터 설명 : 중고차 가격 예측 데이터 (종속변수 :G3)
- 데이터 출처 : https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?select=vw.csv (참고, 데이터 수정)    
- x_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv
- y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv
- y_test(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv

#### 0. 시험 환경 세팅

In [1]:
import pandas as pd

X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv")

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4960, 10), (4960, 2), (2672, 10), (2672, 2))

#### 1. 라이브러리 및 데이터 호출

In [2]:
import numpy as np
import pandas as pd

In [3]:
X_train.drop('carID', axis=1, inplace=True)
carID = X_test.pop('carID')

y = y_train['price']

X_train.shape, X_test.shape, y.shape

((4960, 9), (2672, 9), (4960,))

#### 2. EDA

In [4]:
print(X_train.info())
print('='*50)
print(X_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         4960 non-null   object 
 1   model         4960 non-null   object 
 2   year          4960 non-null   int64  
 3   transmission  4960 non-null   object 
 4   mileage       4960 non-null   int64  
 5   fuelType      4960 non-null   object 
 6   tax           4960 non-null   float64
 7   mpg           4960 non-null   float64
 8   engineSize    4960 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 348.9+ KB
None
brand           0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64


In [5]:
print(X_test.info())
print('='*50)
print(X_test.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         2672 non-null   object 
 1   model         2672 non-null   object 
 2   year          2672 non-null   int64  
 3   transmission  2672 non-null   object 
 4   mileage       2672 non-null   int64  
 5   fuelType      2672 non-null   object 
 6   tax           2672 non-null   float64
 7   mpg           2672 non-null   float64
 8   engineSize    2672 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 188.0+ KB
None
brand           0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64


In [6]:
int_col = X_train.select_dtypes(exclude='object').columns

for c in int_col:
    print(c, X_train[c].nunique())
    print(np.sort(X_train[c].unique()))
    print(X_train[c].value_counts())
    print('='*50)

year 24
[1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010
 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020]
2019    1359
2017     816
2016     743
2018     534
2015     461
2020     337
2014     254
2013     181
2012      56
2011      51
2009      26
2010      25
2008      20
2007      18
2004      18
2006      15
2005      12
2003      10
2001       9
2002       6
1999       3
1998       2
1997       2
2000       2
Name: year, dtype: int64
mileage 3900
[     1      3      4 ... 185000 212000 259000]
1000     51
10       45
5000     37
3000     32
6000     25
         ..
56501     1
62308     1
95408     1
48297     1
26625     1
Name: mileage, Length: 3900, dtype: int64
tax 41
[  0.  10.  20.  30. 115. 125. 135. 140. 145. 150. 155. 160. 165. 185.
 190. 195. 200. 205. 220. 230. 235. 240. 245. 250. 260. 265. 270. 280.
 290. 300. 305. 315. 325. 330. 515. 535. 540. 555. 565. 570. 580.]
145.0    2236
150.0     428
30.0      419
125.0     315
200.0     258
260.0     1

In [7]:
obj_col = X_train.select_dtypes(include='object').columns

for c in obj_col:
    print(c, X_train[c].nunique())
    print(np.sort(X_train[c].unique()))
    print(X_train[c].value_counts())
    print('='*50)

brand 9
['audi' 'bmw' 'ford' 'hyundi' 'merc' 'skoda' 'toyota' 'vauxhall' 'vw']
merc        790
vw          768
ford        766
bmw         638
hyundi      511
toyota      446
skoda       425
audi        343
vauxhall    273
Name: brand, dtype: int64
model 90
[' 6 Series' ' 7 Series' ' 8 Series' ' A7' ' A8' ' Agila' ' Amarok'
 ' Antara' ' Arteon' ' Avensis' ' Beetle' ' CC' ' CLA Class' ' CLK'
 ' CLS Class' ' Caddy' ' Caddy Life' ' Caddy Maxi Life' ' California'
 ' Camry' ' Caravelle' ' Combo Life' ' Edge' ' Eos' ' Fusion' ' G Class'
 ' GL Class' ' GLB Class' ' GLS Class' ' GT86' ' GTC' ' Galaxy' ' Getz'
 ' Grand C-MAX' ' Grand Tourneo Connect' ' Hilux' ' I40' ' I800' ' IQ'
 ' IX20' ' IX35' ' Jetta' ' KA' ' Kamiq' ' Land Cruiser' ' M Class' ' M2'
 ' M3' ' M4' ' M5' ' M6' ' Mustang' ' PROACE VERSO' ' Prius' ' Puma' ' Q8'
 ' R8' ' RS3' ' RS4' ' RS5' ' RS6' ' Rapid' ' Roomster' ' S Class' ' S3'
 ' S4' ' SLK' ' SQ5' ' SQ7' ' Santa Fe' ' Scala' ' Scirocco' ' Shuttle'
 ' Supra' ' Tiguan Allspac

#### 3. Preprocessing

> tax 컬럼 제거

In [8]:
X_train.drop(['tax'], axis=1, inplace=True)
X_test.drop(['tax'], axis=1, inplace=True)

int_col = X_train.select_dtypes(exclude='object').columns
obj_col = X_train.select_dtypes(include='object').columns

> 인코딩 및 표준화
- 범주형 컬럼 : LabelEncoder
- 전체 컬럼 : 표준화

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
X_train[obj_col] = X_train[obj_col].apply(le.fit_transform)
X_test[obj_col] = X_test[obj_col].apply(le.fit_transform)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### 4. Modeling

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.3, shuffle=True, random_state=156)

In [12]:
rf = RandomForestRegressor(random_state=156)
gb = GradientBoostingRegressor(random_state=156)
xgb = XGBRegressor(random_state=156)
xgbrf = XGBRFRegressor(random_state=156)

models = [rf, gb, xgb, xgbrf]

In [13]:
for model in models:
    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)
    
    print(str(model).split('(')[0])
    print('rmse :', np.sqrt(mean_squared_error(y_val, pred)) )
    print('r2 :', r2_score(y_val, pred))
    print('='*10)

RandomForestRegressor
rmse : 3185.6473698089794
r2 : 0.9557165213459863
GradientBoostingRegressor
rmse : 4529.1002009808335
r2 : 0.9104902825704562
XGBRegressor
rmse : 3134.132289494261
r2 : 0.9571371567906148
XGBRFRegressor
rmse : 5131.194093714522
r2 : 0.8851097396947452


In [14]:
xgb = XGBRegressor(random_state=156)

xgb.fit(X_train, y)
pred = xgb.predict(X_test)

In [15]:
out_data = pd.DataFrame({'carID':carID, 'price':pred})
out_data

,carID,price
0,12000,39778.835938
1,12001,24770.234375
2,12004,58415.011719
3,12013,15132.341797
4,12017,49248.867188
...,...,...
2667,19618,33220.523438
2668,19620,17011.599609
2669,19626,21037.207031
2670,19630,23786.392578


In [16]:
# out_data.to_csv('123456.csv', index=False)

In [17]:
# 가채점
print('rmse :', np.sqrt(mean_squared_error(y_test['price'], pred)))
print('r2 :', r2_score(y_test['price'], pred))

rmse : 3705.527094014697
r2 : 0.9501565797805308


#### 5. 다른 모델 평가

In [18]:
rf = RandomForestRegressor(random_state=156)
gb = GradientBoostingRegressor(random_state=156)
xgb = XGBRegressor(random_state=156)
xgbrf = XGBRFRegressor(random_state=156)

models = [rf, gb, xgb, xgbrf]

In [19]:
for model in models:
    model.fit(X_train, y)
    pred = model.predict(X_test)
    
    print(str(model).split('(')[0])
    print('rmse :', np.sqrt(mean_squared_error(y_test['price'], pred)))
    print('r2 :', r2_score(y_test['price'], pred))
    print('='*10)

RandomForestRegressor
rmse : 3937.503360512337
r2 : 0.9437205655743645
GradientBoostingRegressor
rmse : 5114.6415422941345
r2 : 0.9050405363469172
XGBRegressor
rmse : 3705.527094014697
r2 : 0.9501565797805308
XGBRFRegressor
rmse : 5631.521735865332
r2 : 0.8848777208921194


#### 6. pd.get_dummies만 활용
- https://www.kaggle.com/code/yuyuyuyuy/rf-test-r2-score-0-956

In [20]:
import pandas as pd

X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv")

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4960, 10), (4960, 2), (2672, 10), (2672, 2))

In [21]:
y = y_train['price']

X_train_dum = pd.get_dummies(X_train.iloc[:, 1:])
X_test_dum = pd.get_dummies(X_test.iloc[:, 1:])

In [22]:
miss_col = list(set(X_train_dum) - set(X_test_dum))
X_test_dum[miss_col] = 0

X_test_dum = X_test_dum[X_train_dum.columns] # train에만 있는 컬럼 추가한다음에, 컬럼 순서를 맞추기 위해 사용

In [23]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_dum, y, test_size=0.3, shuffle=True, random_state=156)

In [24]:
rf = RandomForestRegressor(random_state=156)
gb = GradientBoostingRegressor(random_state=156)
xgb = XGBRegressor(random_state=156)
xgbrf = XGBRFRegressor(random_state=156)

models = [rf, gb, xgb, xgbrf]

In [25]:
for model in models:
    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)
    
    print(str(model).split('(')[0])
    print('rmse :', np.sqrt(mean_squared_error(y_val, pred)) )
    print('r2 :', r2_score(y_val, pred))
    print('='*10)

RandomForestRegressor
rmse : 3223.5831092802546
r2 : 0.9546555570443112
GradientBoostingRegressor
rmse : 4134.144876727065
r2 : 0.9254208014602556
XGBRegressor
rmse : 3023.0158656652625
r2 : 0.9601225681949662
XGBRFRegressor
rmse : 5130.054311383956
r2 : 0.885160774733543


In [26]:
for model in models:
    model.fit(X_train_dum, y)
    pred = model.predict(X_test_dum)
    
    print(str(model).split('(')[0])
    print('rmse :', np.sqrt(mean_squared_error(y_test['price'], pred)))
    print('r2 :', r2_score(y_test['price'], pred))
    print('='*10)

RandomForestRegressor
rmse : 3260.023863646698
r2 : 0.9614211384338495
GradientBoostingRegressor
rmse : 4528.807235393141
r2 : 0.9255481453307018
XGBRegressor
rmse : 3472.578620915547
r2 : 0.9562264243145023
XGBRFRegressor
rmse : 5327.248700782491
r2 : 0.8969818387981147
